In [2]:
import os
import numpy as np
import pandas as pd
from time import time
from utils import aspect_tools

data_dir = "/home/stavros/DATA/AirbnbReviews"
#data_dir = "D:/"

In [4]:
area = "nyc"
area_dir = os.path.join(data_dir, area)

reviews = pd.read_pickle(os.path.join(area_dir, "reviews_with_aspects_200000samples.pkl"))
print(reviews.shape)
reviews = reviews[pd.notnull(reviews["processed_comments"])]
print(reviews.shape)

(200000, 8)
(200000, 8)


In [5]:
reviews["listing_id"].value_counts()

16276632    113
891117      112
9145202     109
834190      109
10186192    108
           ... 
638894        1
24358122      1
31178681      1
33528562      1
31671911      1
Name: listing_id, Length: 27849, dtype: int64

In [6]:
reviews = pd.read_csv(os.path.join(area_dir, "reviews.csv.gz"))
print(reviews.shape)
reviews = reviews[pd.notnull(reviews["listing_id"])]
reviews = reviews[pd.notnull(reviews["comments"])]
print(reviews.shape)

(1166689, 6)
(1166096, 6)


In [7]:
reviews["listing_id"].value_counts()

9145202     654
903972      615
891117      608
10101135    603
8168619     566
           ... 
31286946      1
36446225      1
8972807       1
34394888      1
22085754      1
Name: listing_id, Length: 38723, dtype: int64

In [8]:
area_dir = os.path.join(data_dir, "athens")
reviews = pd.read_csv(os.path.join(area_dir, "reviews.csv.gz"))
print(reviews.shape)
reviews = reviews[pd.notnull(reviews["listing_id"])]
reviews = reviews[pd.notnull(reviews["comments"])]
print(reviews.shape)

(379303, 6)
(379118, 6)


In [12]:
import collections
reviews_per_listing = collections.Counter({k: v for k, v in reviews["listing_id"].value_counts().items()})

In [15]:
reviews_per_listing.most_common(100)

[(1177492, 675),
 (1079291, 603),
 (2306865, 578),
 (6101639, 534),
 (3431705, 519),
 (2305770, 519),
 (1484797, 516),
 (1746907, 506),
 (7215898, 484),
 (1751520, 464),
 (5025556, 462),
 (3462705, 460),
 (526273, 458),
 (28186, 436),
 (724485, 432),
 (401423, 417),
 (4289381, 416),
 (2579525, 410),
 (80598, 410),
 (13553080, 405),
 (702480, 405),
 (10400201, 396),
 (2883333, 395),
 (2428866, 393),
 (306234, 392),
 (14081186, 390),
 (14583913, 382),
 (59663, 374),
 (5732674, 369),
 (727374, 369),
 (10898803, 365),
 (11403148, 365),
 (9337716, 359),
 (6101477, 356),
 (13145534, 356),
 (16278854, 354),
 (661314, 348),
 (11669024, 347),
 (1529526, 345),
 (5266430, 345),
 (17495305, 344),
 (9291989, 344),
 (5788192, 340),
 (3759135, 340),
 (2487816, 339),
 (2513822, 338),
 (7797582, 338),
 (1207614, 337),
 (6125281, 334),
 (918812, 334),
 (15841192, 333),
 (5725222, 332),
 (1094063, 329),
 (905315, 328),
 (2240132, 328),
 (783657, 325),
 (2299339, 325),
 (6622262, 323),
 (3630518, 323),
 (